In [1]:
%matplotlib inline

import sys
import os
import pathlib
sys.path.append('/g/data/mn51/users/mtb563/toolbox/modules')
import utils
import xarray as xr

xr.set_options(keep_attrs=True)

# User defined
indices = ['HWAtx','HWF','HWD','HWN']
bc_methods = ['raw']
pathways = ['ssp370']
odir_parent = '/g/data/ia39/ncra/heat/data/'

#Data specific information
training_data = 'AGCD-1960-2022'
project = 'CORDEX-CMIP6'
grid='AGCD-05i'
data_status = 'test-data'
data_version = 'v1-r1'

rcm = {'BOM':'BARPA-R','CSIRO':'CCAM-v2203-SN'}


In [2]:
def get_fpath_infile():
    fdir = f"/g/data/mn51/projects/work_package_4/climate_hazard_indices/heat/{data_status}/{project}/{grid}/{institution}/{gcm}/{pathway}/{member}/{rcm[institution]}/{data_version}/{bc_method}/{bc_training_data}/day/{index_fname1}/"
    fname = f"{index_fname2}_{grid}_{gcm}_{pathway}_{member}_{institution}_{rcm[institution]}_{data_version}{'-'+bc_method+'-'+bc_training_data if bc_method != 'raw' else ''}_{tperiod}.nc"
    return fdir+fname

def get_dirpath_gcms():
    """returns the directory path within which the data is separated per GCM"""
    return f"/g/data/mn51/projects/work_package_4/climate_hazard_indices/heat/{data_status}/{project}/{grid}/{institution}/"

def construct_ensemble_stats(method):
    assert method in ['MME','MME-change']
            
    members = []
    GCM_RCM_abbr = []
    for e in ensemble_members:
        ddir = odir_avg if method == 'MME' else odir_change
        infile = ddir+f"{index}_{grid}_{e[0]}_{pathway}_{e[2]}_{e[3]}_{e[1]}_{data_version}{'-'+bc_method+'-'+bc_training_data if bc_method != 'raw' else ''}_GWL{int(gwl*10)}{'-avg' if method == 'MME' else f'-GWL{int(gwl_ref*10)}-change'}.nc"
        da = utils.generalio.read_data(infiles=[infile],var=index)
        #da = da.expand_dims(dim={'model':f"{e[0]}-{e[1]}"},axis=0)
        members.append(da)
        GCM_RCM_abbr.append(f"{e[0]}-{e[1]}")
            
    #ensemble = xr.merge(members).to_dataset()
    ensemble = xr.concat(members, dim='member').to_dataset()
    
    MME50 = ensemble.quantile(q=0.50,dim='member',keep_attrs=True).squeeze()
    #MME50.attrs = {}
            
    MME10 = ensemble.quantile(q=0.10,dim='member',keep_attrs=True).squeeze()
    #MME10.attrs = {}

    MME90 = ensemble.quantile(q=0.90,dim='member',keep_attrs=True).squeeze()
    #MME90.attrs = {}

    if method == 'MME':
        MME50.attrs['description'] = '50th percentile of inter-model spread of: '+MME50[index].attrs['long_name']+f" averaged over GWL {gwl} degC"
        MME10.attrs['description'] = '10th percentile of inter-model spread of: '+MME10[index].attrs['long_name']+f" averaged over GWL {gwl} degC"
        MME90.attrs['description'] = '90th percentile of inter-model spread of: '+MME90[index].attrs['long_name']+f" averaged over GWL {gwl} degC"
        odir_mme = odir_parent+f"{index}/{'bias-corrected' if bc_method != 'raw' else 'raw'}/ensemble/GWL-average/"
    elif method == 'MME-change':
        MME50.attrs['description'] = '50th percentile of inter-model spread of: average difference in '+MME50[index].attrs['long_name']+f" between GWL {gwl} degC and GWL {gwl_ref} degC"
        MME10.attrs['description'] = '10th percentile of inter-model spread of: average difference in '+MME10[index].attrs['long_name']+f" between GWL {gwl} degC and GWL {gwl_ref} degC"
        MME90.attrs['description'] = '90th percentile of inter-model spread of: average difference in '+MME90[index].attrs['long_name']+f" between GWL {gwl} degC and GWL {gwl_ref} degC"
        odir_mme = odir_parent+f"{index}/{'bias-corrected' if bc_method != 'raw' else 'raw'}/ensemble/GWL-change/"
        
    for d in [MME50,MME10,MME90]:
        d.attrs['models'] = GCM_RCM_abbr
        d.attrs['pathway'] = pathway
        d.attrs['GWL'] = f"GWL {gwl} degC" if method == 'MME' else f"GWL {gwl} degC - GWL {gwl_ref} degC"
        d.attrs['bias_correction_method'] = bc_method
        d.attrs['contact'] = "Mitchell Black (mitchell.black@bom.gov.au)"
        d.attrs['code'] = "https://github.com/AusClimateService/hazards-heat"  
    
    pathlib.Path(odir_mme).mkdir(parents=True, exist_ok=True)
            
    utils.generalio.save_data(MME50,ofile=odir_mme+f"{index}_{grid}_MME50_{pathway}_{data_version}{'-'+bc_method+'-'+bc_training_data if bc_method != 'raw' else ''}_GWL{int(gwl*10)}{f'-GWL{int(gwl_ref*10)}-change' if method == 'MME-change' else ''}.nc")                       
    utils.generalio.save_data(MME10,ofile=odir_mme+f"{index}_{grid}_MME10_{pathway}_{data_version}{'-'+bc_method+'-'+bc_training_data if bc_method != 'raw' else ''}_GWL{int(gwl*10)}{f'-GWL{int(gwl_ref*10)}-change' if method == 'MME-change' else ''}.nc")    
    utils.generalio.save_data(MME90,ofile=odir_mme+f"{index}_{grid}_MME90_{pathway}_{data_version}{'-'+bc_method+'-'+bc_training_data if bc_method != 'raw' else ''}_GWL{int(gwl*10)}{f'-GWL{int(gwl_ref*10)}-change' if method == 'MME-change' else ''}.nc")    

    del(members,GCM_RCM_abbr,ensemble,MME50,MME10,MME90)

In [3]:
for index in indices:
    for bc_method in bc_methods:

        ensemble_members = []
        
        if bc_method == 'raw':
            bc_training_data = 'raw'
        else:
            bc_training_data = training_data
        
        # Construct GWL information for individual GCM-RCM combinations
        
        for institution in ['BOM','CSIRO']:
            gcms = os.listdir(get_dirpath_gcms())
            for gcm in gcms:
                for pathway in pathways:
                    for member in os.listdir(f'{get_dirpath_gcms()}/{gcm}/{pathway}'):

                        # Construct GWL timeslices and averages per model
                        
                        if index in ['HWAtx','HWF','HWD','HWN']:
                            index_fname1 = 'EHF'
                            index_fname2 = 'EHF-metrics'
                            tperiod = 'annual_19800701-20990630_Jul2Jun'
                        else:
                            index_fname1 = index
                            index_fname2 = index
                            tperiod = 'annual_19800101-20991231'
                        
                        ds = utils.generalio.read_data(infiles=[get_fpath_infile()],var=index)

                        for gwl in [1.2,1.5,2.0,3.0]:
                            
                            gwl_ts = utils.gwl.get_GWL_timeslice(ds,CMIP=project.replace('CORDEX-',''),GCM=gcm,ensemble=member,pathway=pathway,GWL=gwl)
                            gwl_avg = gwl_ts.mean(dim='time',keep_attrs=True)
                            
                            gwl_syear, gwl_eyear = utils.gwl.get_GWL_syear_eyear(CMIP=project.replace('CORDEX-',''),GCM=gcm,ensemble=member,pathway=pathway,GWL=gwl)
                           
                            gwl_ts = gwl_ts.to_dataset()
                            gwl_ts.attrs['description'] = gwl_ts[index].attrs['long_name']+f" for the 20-year period correspongig with GWL{gwl} degC ({gwl_syear}-{gwl_eyear}; {pathway})"
                            
                            gwl_avg = gwl_avg.to_dataset()
                            gwl_avg.attrs['description'] = gwl_avg[index].attrs['long_name']+f" averaged over the 20-year period correspongig with GWL{gwl} degC ({gwl_syear}-{gwl_eyear}; {pathway})"

                            for d in [gwl_ts, gwl_avg]:
                                d.attrs['driving_model'] = gcm
                                d.attrs['downscaling_model'] = rcm[institution]
                                d.attrs['pathway'] = pathway
                                d.attrs['GWL'] = f"GWL {gwl} degC"
                                d.attrs['bias_correction_method'] = bc_method
                                d.attrs['contact'] = "Mitchell Black (mitchell.black@bom.gov.au)"
                                d.attrs['code'] = "https://github.com/AusClimateService/hazards-heat"                           

                            odir_ts = odir_parent+f"{index}/{'bias-corrected' if bc_method != 'raw' else 'raw'}/individual_models/GWL-timeseries/"
                            pathlib.Path(odir_ts).mkdir(parents=True, exist_ok=True)
                            utils.generalio.save_data(gwl_ts,ofile=odir_ts+f"{index}_{grid}_{gcm}_{pathway}_{member}_{institution}_{rcm[institution]}_{data_version}{'-'+bc_method+'-'+bc_training_data if bc_method != 'raw' else ''}_GWL{int(gwl*10)}.nc")
                            
                            odir_avg = odir_parent+f"{index}/{'bias-corrected' if bc_method != 'raw' else 'raw'}/individual_models/GWL-average/"
                            pathlib.Path(odir_avg).mkdir(parents=True, exist_ok=True)
                            utils.generalio.save_data(gwl_avg,ofile=odir_avg+f"{index}_{grid}_{gcm}_{pathway}_{member}_{institution}_{rcm[institution]}_{data_version}{'-'+bc_method+'-'+bc_training_data if bc_method != 'raw' else ''}_GWL{int(gwl*10)}-avg.nc")                      

                            del(gwl_ts,gwl_avg)
                            
                        # Construct GWL-change fields per model
                        gwl_ref = 1.2
                        ref = utils.generalio.read_data(infiles=[odir_avg+f"{index}_{grid}_{gcm}_{pathway}_{member}_{institution}_{rcm[institution]}_{data_version}{'-'+bc_method+'-'+bc_training_data if bc_method != 'raw' else ''}_GWL{int(gwl_ref*10)}-avg.nc"],var=index)
                        
                        for gwl in [1.5,2.0,3.0]:
                            future = utils.generalio.read_data(infiles=[odir_avg+f"{index}_{grid}_{gcm}_{pathway}_{member}_{institution}_{rcm[institution]}_{data_version}{'-'+bc_method+'-'+bc_training_data if bc_method != 'raw' else ''}_GWL{int(gwl*10)}-avg.nc"],var=index)
                            gwl_change = future - ref
                            gwl_change = gwl_change.to_dataset()
                            gwl_change.attrs = {}
                            gwl_change.attrs['description'] = 'difference in average '+gwl_change[index].attrs['long_name']+f" between GWL {gwl} degC and GWL {gwl_ref} degC)"
                            gwl_change.attrs['driving_model'] = gcm
                            gwl_change.attrs['downscaling_model'] = rcm[institution]
                            gwl_change.attrs['pathway'] = pathway
                            gwl_change.attrs['GWL'] = f"GWL {gwl} degC - GWL {gwl_ref} degC"
                            gwl_change.attrs['bias_correction_method'] = bc_method
                            gwl_change.attrs['contact'] = "Mitchell Black (mitchell.black@bom.gov.au)"
                            gwl_change.attrs['code'] = "https://github.com/AusClimateService/hazards-heat"  
                            
                            odir_change = odir_parent+f"{index}/{'bias-corrected' if bc_method != 'raw' else 'raw'}/individual_models/GWL-change/"
                            pathlib.Path(odir_change).mkdir(parents=True, exist_ok=True)
                            utils.generalio.save_data(gwl_change,ofile=odir_change+f"{index}_{grid}_{gcm}_{pathway}_{member}_{institution}_{rcm[institution]}_{data_version}{'-'+bc_method+'-'+bc_training_data if bc_method != 'raw' else ''}_GWL{int(gwl*10)}-GWL{int(gwl_ref*10)}-change.nc")                       
                            del(gwl_change)

                        ensemble_members.append([gcm,rcm[institution],member,institution])

        # Construct ensemble-aggregate GWL fields

        for gwl in [1.2, 1.5, 2.0, 3.0]:
            construct_ensemble_stats('MME')

        # Construct ensemble-aggregate GWL-change fields
        for gwl in [1.5, 2.0, 3.0]:
            construct_ensemble_stats('MME-change')
 
                                                